In [1]:
from langchain.document_loaders import UnstructuredFileLoader

files = ["report.docx"]
docs = []

for file in files:
    loader = UnstructuredFileLoader(file)
    docs.extend(loader.load())


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(docs)


In [ ]:
from langchain.embeddings import OpenAIEmbeddings
import os
import getpass
os.environ["OPENAI_API_KEY"]=getpass.getpass()


In [4]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embedding)
vectorstore.save_local("rag_index")


In [5]:
vectorstore = FAISS.load_local("rag_index", embedding)


In [8]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo")
qa = RetrievalQA.from_chain_type(llm=llm,
                                  retriever=vectorstore.as_retriever(),
                                  chain_type="stuff")

query = " The Treaty of Tordesillas (1494) divided the world between two empires.  Name the two empires, then briefly describe the differences between their styles of imperialism and the resultant long-term differences in their impact on local peoples.   "
response = qa.run(query)
print(response)


The Treaty of Tordesillas divided the world between Spain and Portugal. Spain engaged in settler colonialism, involving the exportation of a large population from the home country, conquest, and subjugation of local populations, and the establishment of new colonial governments. This led to the widespread use of the Spanish language and a strong connection with European political developments, such as nationalism, which eventually influenced their colonies' push for independence.

Portugal, on the other hand, practiced trader or "elite" colonialism, where it established ties with favored local rulers to secure trading rights and safe ports in exchange for support. This strategy led to the creation of new trading routes and patterns across Asia but did not have a significant cultural influence on the local populations. The local elites, often educated in the metropole, would later adopt national principles.
